<a href="https://colab.research.google.com/github/granantuin/Model_vs_data_label/blob/master/randomforest.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from sklearn.metrics import precision_recall_fscore_support,confusion_matrix,accuracy_score
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split,cross_val_score,cross_validate,GridSearchCV
from sklearn.dummy import DummyClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.utils import resample
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import RandomizedSearchCV

In [2]:
from google.colab import drive
drive.mount('/content/drive')


Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
def evaluate(y_test,y_pred):
  index=["E","N","NE","NW","S","SE","SW","W"]
  print(pd.DataFrame(confusion_matrix(y_test, y_pred), index=index, columns=index))
  print("****************")
  print("Accuracy=","{:.2%}".format(accuracy_score(y_test, y_pred)),"// Model Accuracy=45%")
  results= precision_recall_fscore_support(y_test, y_pred, average=None, )
  df=pd.DataFrame({"Precision":results[0],"Recall":results[1],"F1":results[2],"W_DIR":index})
  df=df.set_index("W_DIR")
  print("Average precision =","{:.2%}".format(df["Precision"].mean()),"// Model precision=28%")
  print("Average recall =","{:.2%}".format(df["Recall"].mean()),"// Model recall=28%")
  results= precision_recall_fscore_support(y_test, y_pred, average="micro", )
  print("Precision micro=","{:.2%}".format(results[0]),"//Model precision micro=45%")
  print("Recall micro=","{:.2%}".format(results[1]),"//Model recall micro=45%")
  results= precision_recall_fscore_support(y_test, y_pred, average='weighted', )
  print("Precision weighted=","{:.2%}".format(results[0]),"//Model weighted=56%")
  print("Recall weighted =","{:.2%}".format(results[1]),"//Model weighted=53%")
  print("****************")
  print(df)

In [0]:
drive_4km="/content/drive/My Drive/Colab Notebooks/model_vs_data/Coron_4km_h24toh48_dir/"
drive_1km="/content/drive/My Drive/Colab Notebooks/model_vs_data/Coron_Mars_H24to48_dir/"
dr=[drive_4km,drive_1km]
y_data=pd.read_excel(dr[1]+"y_data.xlsx")
x_data=pd.read_csv(dr[1]+"x_data",usecols=range(1,10))
y_sp=pd.read_excel(dr[1]+"y_sp.xlsx") # units=Km/h to m/s
y_sp["spd"]=y_sp["spd"]/3.6

In [0]:
NE=y_data[(y_data["value"]>22.5) & (y_data["value"]<67.5)]
NE["label"]="NE"
E=y_data[(y_data["value"]>67.5) & (y_data["value"]<112.5)]
E["label"]="E"
SE=y_data[(y_data["value"]>112.5) & (y_data["value"]<157.5)]
SE["label"]="SE"
S=y_data[(y_data["value"]>157.5) & (y_data["value"]<202.5)]
S["label"]="S"
SW=y_data[(y_data["value"]>202.5) & (y_data["value"]<247.5)]
SW["label"]="SW"
W=y_data[(y_data["value"]>247.5) & (y_data["value"]<292.5)]
W["label"]="W"
NW=y_data[(y_data["value"]>292.5) & (y_data["value"]<337.5)]
NW["label"]="NW"
N=y_data[(y_data["value"]>337.5) | (y_data["value"]<22.5)]
N["label"]="N"
winds=[NE,N,E,SE,S,SW,W,NW]
y_data=pd.concat(winds)
y_data=y_data.sort_values(by="date")

**Dummy class**

In [62]:
#strategy=["stratified","most_frequent”]
#“stratified”: generates predictions by respecting the training set’s class distribution
#“most_frequent”: always predicts the most frequent label in the training set.
x_train, x_test, y_train, y_test = train_test_split(x_data,y_data["label"], test_size=0.2, random_state=5)
y_pred = DummyClassifier(strategy="stratified").fit(x_train, y_train).predict(x_test)
evaluate(y_test,y_pred)
x_train, x_test, y_train, y_test = train_test_split(x_data,y_data["label"], test_size=0.2, random_state=5)
y_pred = DummyClassifier(strategy="most_frequent").fit(x_train, y_train).predict(x_test)
evaluate(y_test,y_pred)

    E  N  NE  NW  S  SE  SW   W
E   0  0   2   0  0   0   2   0
N   0  0   4   0  0   0   2   1
NE  0  5  32   1  4   0   9  11
NW  0  1   1   1  1   0   0   1
S   0  0   2   0  0   0   0   1
SE  0  0   1   0  0   0   0   0
SW  0  1  11   0  5   1   3   2
W   1  0  12   0  1   0   1   1
****************
Accuracy= 30.58% // Model Accuracy=45%
Average precision = 15.35% // Model precision=28%
Average recall = 11.36% // Model recall=28%
Precision micro= 30.58% //Model precision micro=45%
Recall micro= 30.58% //Model recall micro=45%
Precision weighted= 31.42% //Model weighted=56%
Recall weighted = 30.58% //Model weighted=53%
****************
       Precision    Recall        F1
W_DIR                               
E       0.000000  0.000000  0.000000
N       0.000000  0.000000  0.000000
NE      0.492308  0.516129  0.503937
NW      0.500000  0.200000  0.285714
S       0.000000  0.000000  0.000000
SE      0.000000  0.000000  0.000000
SW      0.176471  0.130435  0.150000
W       0.058824  0.

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


**Random Forest Class**

In [50]:
x_train, x_test, y_train, y_test = train_test_split(x_data,y_data["label"], test_size=0.2, random_state=5)
y_pred = RandomForestClassifier(n_estimators=40).fit(x_train, y_train).predict(x_test)
evaluate(y_test,y_pred)

    E  N  NE  NW  S  SE  SW  W
E   3  0   3   0  1   0   0  1
N   1  5   5   0  0   0   1  3
NE  0  3  65   0  1   0   5  2
NW  1  0   1   0  0   0   3  4
S   1  0   1   0  1   0   3  2
SE  0  1   3   0  3   0   2  1
SW  2  0   4   0  0   0  11  4
W   0  0   6   1  0   0   4  6
****************
Accuracy= 55.49% // Model Accuracy=45%
Average precision = 30.95% // Model precision=28%
Average recall = 32.07% // Model recall=28%
Precision micro= 55.49% //Model precision micro=45%
Recall micro= 55.49% //Model recall micro=45%
Precision weighted= 49.51% //Model weighted=56%
Recall weighted = 55.49% //Model weighted=53%
****************
       Precision    Recall        F1
W_DIR                               
E       0.375000  0.375000  0.375000
N       0.555556  0.333333  0.416667
NE      0.738636  0.855263  0.792683
NW      0.000000  0.000000  0.000000
S       0.166667  0.125000  0.142857
SE      0.000000  0.000000  0.000000
SW      0.379310  0.523810  0.440000
W       0.260870  0.352941  0

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


**Synthetic samples**

In [42]:
x_train, x_test, y_train, y_test = train_test_split(x_data, y_data["label"], test_size=0.4, random_state=27)
print("Original train samples=",x_train.shape[0])
sm = SMOTE(random_state=27,)
x_train, y_train = sm.fit_sample(x_train, y_train)
print("Synthetic train samples=",x_train.shape[0])
print("****Randomforest****")
y_pred = RandomForestClassifier(n_estimators=40).fit(x_train, y_train).predict(x_test)
evaluate(y_test,y_pred)

Original train samples= 489
Synthetic train samples= 1944
****Randomforest****
     E   N  NE  NW  S  SE  SW   W
E    4   3  10   0  0   3   4   0
N    1   9  12   0  0   1   0   0
NE  10  11  98   2  2   6   2   2
NW   1   2   2   4  0   1   4   0
S    0   2   2   2  8   3   7   0
SE   4   1   1   2  0   3   2   0
SW   0   3   4   3  7   6  22   6
W    1   5   9   5  6   2   7  10
****************
Accuracy= 48.32% // Model Accuracy=45%
Average precision = 35.68% // Model precision=28%
Average recall = 34.98% // Model recall=28%
Precision micro= 48.32% //Model precision micro=45%
Recall micro= 48.32% //Model recall micro=45%
Precision weighted= 50.81% //Model weighted=56%
Recall weighted = 48.32% //Model weighted=53%
****************
       Precision    Recall        F1
W_DIR                               
E       0.190476  0.166667  0.177778
N       0.250000  0.391304  0.305085
NE      0.710145  0.736842  0.723247
NW      0.222222  0.285714  0.250000
S       0.347826  0.333333  0.3404

**Filtering wind speed thershold**

In [0]:

x_data["date"]=y_data["date"]
x_data=x_data.set_index("date")
y_data=y_data.set_index("date")
y_sp=y_sp.set_index("date")
tot=pd.concat([x_data,y_data,y_sp],axis=1).dropna()
x_data=tot[tot["spd"]>2].iloc[:, 0:9]
y_data=tot[tot["spd"]>2].iloc[:, 10:11]


In [52]:
x_train, x_test, y_train, y_test = train_test_split(x_data,y_data["label"], test_size=0.4, random_state=5)
y_pred = RandomForestClassifier(n_estimators=40).fit(x_train, y_train).predict(x_test)
evaluate(y_test,y_pred)

    E  N   NE  NW  S  SE  SW   W
E   1  0    5   0  1   0   1   1
N   0  3    5   0  1   0   1   1
NE  0  4  113   0  0   0   1   3
NW  1  2    0   1  0   0   0   4
S   0  0    1   0  2   0   2   1
SE  0  0    0   0  0   0   0   1
SW  0  1    4   1  6   1  25  10
W   0  0    6   2  2   0  16  12
****************
Accuracy= 64.88% // Model Accuracy=45%
Average precision = 37.09% // Model precision=28%
Average recall = 32.66% // Model recall=28%
Precision micro= 64.88% //Model precision micro=45%
Recall micro= 64.88% //Model recall micro=45%
Precision weighted= 63.12% //Model weighted=56%
Recall weighted = 64.88% //Model weighted=53%
****************
       Precision    Recall        F1
W_DIR                               
E       0.500000  0.111111  0.181818
N       0.300000  0.272727  0.285714
NE      0.843284  0.933884  0.886275
NW      0.250000  0.125000  0.166667
S       0.166667  0.333333  0.222222
SE      0.000000  0.000000  0.000000
SW      0.543478  0.520833  0.531915
W       0.3